In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 1. 기존 리포지토리 폴더로 이동
import os
os.chdir('/content/drive/MyDrive/Codeit_AI_4th_Drug_image_CV_project')

In [3]:
# 2. 경로 확인
!pwd

/content/drive/MyDrive/Codeit_AI_4th_Drug_image_CV_project


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
from collections import defaultdict

In [ ]:
def analyze_prediction_results(csv_path="submission_baseline_return.csv"):
    """RT-DETR 예측 결과를 종합 분석 (COCO 형식 지원)"""

    print(f"=== {csv_path} 분석 시작 ===")

    # 데이터 로드
    try:
        df = pd.read_csv(csv_path)
        print(f"데이터 로드 성공: {len(df)}개 예측")
        print(f"컬럼: {list(df.columns)}")
    except:
        print(f"파일을 찾을 수 없습니다: {csv_path}")
        return None

    # 컬럼명 확인 및 통합 (COCO 형식 지원)
    if 'score' in df.columns:
        df['confidence'] = df['score']  # score -> confidence 통일
    if 'bbox_x' in df.columns and 'bbox_y' in df.columns:
        # COCO 형식 (x,y,w,h) -> (x1,y1,x2,y2) 변환
        df['x1'] = df['bbox_x']
        df['y1'] = df['bbox_y']
        df['x2'] = df['bbox_x'] + df['bbox_w']
        df['y2'] = df['bbox_y'] + df['bbox_h']
        df['box_width'] = df['bbox_w']
        df['box_height'] = df['bbox_h']
        df['box_area'] = df['bbox_w'] * df['bbox_h']

    # 1. 기본 통계
    print(f"\n=== 1. 기본 통계 ===")
    print(f"총 예측 수: {len(df)}")

    if 'image_id' in df.columns:
        unique_images = df['image_id'].nunique()
        print(f"처리된 이미지 수: {unique_images}")
    elif 'image_name' in df.columns:
        unique_images = df['image_name'].nunique()
        print(f"처리된 이미지 수: {unique_images}")

    print(f"탐지된 카테고리 수: {df['category_id'].nunique()}")
    print(f"평균 신뢰도: {df['confidence'].mean():.3f}")
    print(f"신뢰도 범위: {df['confidence'].min():.3f} ~ {df['confidence'].max():.3f}")

    # 2. 신뢰도 분포 분석
    print(f"\n=== 2. 신뢰도 분포 분석 ===")
    confidence_stats = df['confidence'].describe()
    print(confidence_stats)

    # 신뢰도 구간별 분포
    conf_bins = [0, 0.25, 0.3, 0.35, 0.4, 0.5, 0.6, 0.7, 0.8, 1.0]
    conf_counts = pd.cut(df['confidence'], bins=conf_bins, include_lowest=True).value_counts().sort_index()
    print(f"\n신뢰도 구간별 예측 수:")
    for interval, count in conf_counts.items():
        percent = (count / len(df)) * 100
        print(f"  {interval}: {count}개 ({percent:.1f}%)")

    # 3. 카테고리별 분석
    print(f"\n=== 3. 카테고리별 분석 ===")

    image_col = 'image_id' if 'image_id' in df.columns else 'image_name'
    category_stats = df.groupby('category_id').agg({
        'confidence': ['count', 'mean', 'std'],
        image_col: 'nunique'
    }).round(3)

    category_stats.columns = ['예측수', '평균신뢰도', '신뢰도표준편차', '이미지수']
    category_stats = category_stats.sort_values('예측수', ascending=False)

    print(f"상위 10개 카테고리:")
    print(category_stats.head(10))

    print(f"\n하위 10개 카테고리:")
    print(category_stats.tail(10))

    # 클래스 불균형 분석
    max_predictions = category_stats['예측수'].max()
    min_predictions = category_stats['예측수'].min()
    imbalance_ratio = max_predictions / min_predictions
    print(f"\n클래스 불균형 비율: {imbalance_ratio:.1f}:1 (최다 {max_predictions}개 vs 최소 {min_predictions}개)")

    # 4. 이미지별 탐지 수 분석
    print(f"\n=== 4. 이미지별 탐지 수 분석 ===")
    detections_per_image = df.groupby(image_col).size()
    print(f"이미지당 평균 탐지 수: {detections_per_image.mean():.2f}")
    print(f"탐지 수 범위: {detections_per_image.min()} ~ {detections_per_image.max()}")

    # 탐지 수 분포
    detection_counts = detections_per_image.value_counts().sort_index()
    print(f"\n이미지별 탐지 수 분포:")
    for count, images in detection_counts.head(10).items():
        percent = (images / len(detections_per_image)) * 100
        print(f"  {count}개 탐지: {images}개 이미지 ({percent:.1f}%)")

    # 5. 박스 크기 분석
    print(f"\n=== 5. 박스 크기 분석 ===")

    if 'box_area' in df.columns:  # 이미 계산된 경우
        print(f"박스 너비 통계:")
        print(f"  평균: {df['box_width'].mean():.1f}")
        print(f"  범위: {df['box_width'].min():.1f} ~ {df['box_width'].max():.1f}")

        print(f"박스 높이 통계:")
        print(f"  평균: {df['box_height'].mean():.1f}")
        print(f"  범위: {df['box_height'].min():.1f} ~ {df['box_height'].max():.1f}")

        print(f"박스 면적 통계:")
        print(f"  평균: {df['box_area'].mean():.0f}")
        print(f"  범위: {df['box_area'].min():.0f} ~ {df['box_area'].max():.0f}")

    # 6. 문제 패턴 식별
    print(f"\n=== 6. 잠재적 문제 패턴 ===")

    if 'box_area' in df.columns:
        # 매우 작은 박스들 (잠재적 노이즈)
        small_boxes = df[df['box_area'] < 1000]
        print(f"매우 작은 박스 (면적<1000): {len(small_boxes)}개 ({len(small_boxes)/len(df)*100:.1f}%)")

        # 매우 큰 박스들 (잠재적 오탐지)
        large_boxes = df[df['box_area'] > 150000]
        print(f"매우 큰 박스 (면적>150000): {len(large_boxes)}개 ({len(large_boxes)/len(df)*100:.1f}%)")

    # 낮은 신뢰도 예측들
    low_conf = df[df['confidence'] < 0.3]
    print(f"낮은 신뢰도 (0.3미만): {len(low_conf)}개 ({len(low_conf)/len(df)*100:.1f}%)")

    # 과탐지 이미지들 (10개 이상 탐지)
    over_detected = detections_per_image[detections_per_image >= 10]
    print(f"과탐지 이미지 (10개이상): {len(over_detected)}개 ({len(over_detected)/len(detections_per_image)*100:.1f}%)")

    # 7. 개선 방향 제안
    print(f"\n=== 7. 개선 방향 제안 ===")

    # 신뢰도 기반 제안
    conf_current = len(df)
    conf_27_count = len(df[df['confidence'] >= 0.27])
    conf_30_count = len(df[df['confidence'] >= 0.30])
    conf_35_count = len(df[df['confidence'] >= 0.35])

    print(f"임계값별 예측 수 변화:")
    print(f"  현재 (0.25이상): {conf_current}개")
    print(f"  0.27로 상향시: {conf_27_count}개 ({(conf_27_count-conf_current)/conf_current*100:+.1f}%)")
    print(f"  0.30으로 상향시: {conf_30_count}개 ({(conf_30_count-conf_current)/conf_current*100:+.1f}%)")
    print(f"  0.35로 상향시: {conf_35_count}개 ({(conf_35_count-conf_current)/conf_current*100:+.1f}%)")

    # 박스 크기 기반 제안
    if 'box_area' in df.columns:
        area_filtered = df[(df['box_area'] >= 1000) & (df['box_area'] <= 150000)]
        print(f"\n박스 크기 필터링 효과:")
        print(f"  현재: {len(df)}개")
        print(f"  크기 필터링 후: {len(area_filtered)}개 ({(len(area_filtered)-len(df))/len(df)*100:+.1f}%)")

    return df

In [ ]:
def analyze_category_imbalance(df):
    """클래스 불균형 상세 분석"""

    print(f"\n=== 클래스 불균형 상세 분석 ===")

    category_counts = df['category_id'].value_counts().sort_values(ascending=False)

    print(f"총 {len(category_counts)}개 카테고리")
    print(f"최다 카테고리: {category_counts.iloc[0]}개 (ID: {category_counts.index[0]})")
    print(f"최소 카테고리: {category_counts.iloc[-1]}개 (ID: {category_counts.index[-1]})")
    print(f"불균형 비율: {category_counts.iloc[0] / category_counts.iloc[-1]:.1f}:1")

    # 카테고리 그룹별 분류
    very_frequent = category_counts[category_counts >= 100]
    frequent = category_counts[(category_counts >= 20) & (category_counts < 100)]
    rare = category_counts[(category_counts >= 5) & (category_counts < 20)]
    very_rare = category_counts[category_counts < 5]

    print(f"\n카테고리 분포:")
    print(f"  매우 빈번 (100+개): {len(very_frequent)}개 카테고리")
    print(f"  빈번 (20-99개): {len(frequent)}개 카테고리")
    print(f"  희소 (5-19개): {len(rare)}개 카테고리")
    print(f"  매우 희소 (<5개): {len(very_rare)}개 카테고리")

    return category_counts

In [ ]:
def suggest_improvement_strategies(df):
    """분석 결과 기반 개선 전략 제안"""

    print(f"\n=== 📈 구체적 개선 전략 제안 ===")

    # 현재 상황 요약
    total_preds = len(df)
    avg_conf = df['confidence'].mean()
    low_conf_count = len(df[df['confidence'] < 0.3])
    small_box_count = len(df[df['box_area'] < 100])

    print(f"현재 상황:")
    print(f"  - 총 예측: {total_preds}개")
    print(f"  - 평균 신뢰도: {avg_conf:.3f}")
    print(f"  - 낮은 신뢰도(<0.3): {low_conf_count}개 ({low_conf_count/total_preds*100:.1f}%)")
    print(f"  - 작은 박스(<100): {small_box_count}개 ({small_box_count/total_preds*100:.1f}%)")

    # 전략별 예상 효과
    print(f"\n🎯 추천 개선 전략 (우선순위순):")

    # 전략 1: 후처리 필터링
    filtered_df = df[(df['confidence'] >= 0.27) & (df['box_area'] >= 50)]
    strategy1_reduction = total_preds - len(filtered_df)
    print(f"\n1. 후처리 필터링 (신뢰도≥0.27, 면적≥50)")
    print(f"   - 제거 예측: {strategy1_reduction}개 ({strategy1_reduction/total_preds*100:.1f}%)")
    print(f"   - 남은 예측: {len(filtered_df)}개")
    print(f"   - 위험도: ⭐ (매우 낮음)")
    print(f"   - 소요시간: 1분")

    # 전략 2: NMS 강화
    print(f"\n2. NMS 파라미터 조정")
    print(f"   - 중복 제거 강화로 품질 향상")
    print(f"   - 예상 효과: 5-10% 예측 수 감소")
    print(f"   - 위험도: ⭐⭐ (낮음)")
    print(f"   - 소요시간: 5분")

    # 전략 3: 카테고리별 임계값
    category_conf = df.groupby('category_id')['confidence'].mean().sort_values()
    low_conf_categories = category_conf[category_conf < 0.4]
    print(f"\n3. 카테고리별 차별화된 임계값")
    print(f"   - 저신뢰도 카테고리: {len(low_conf_categories)}개")
    print(f"   - 이들만 높은 임계값 적용")
    print(f"   - 위험도: ⭐⭐⭐ (중간)")
    print(f"   - 소요시간: 30분")

    return filtered_df

In [ ]:
def run_comprehensive_analysis(csv_path="submission_baseline.csv"):
    """종합적인 데이터 분석 실행"""

    # 기본 분석
    df = analyze_prediction_results(csv_path)

    if df is not None:
        # 클래스 불균형 분석
        category_counts = analyze_category_imbalance(df)

        # 개선 전략 제안
        filtered_df = suggest_improvement_strategies(df)

        return df, category_counts, filtered_df

    return None, None, None

In [ ]:
if __name__ == "__main__":
    # 종합 분석 실행
    df, category_counts, filtered_df = run_comprehensive_analysis("./submission_baseline_return.csv")

    print(f"\n=== 분석 완료 ===")
    print(f"다음 단계: 제안된 개선 전략 중 1번(후처리 필터링)부터 시작 권장")

=== ./submission_baseline_return.csv 분석 시작 ===
데이터 로드 성공: 3307개 예측
컬럼: ['annotation_id', 'image_id', 'category_id', 'bbox_x', 'bbox_y', 'bbox_w', 'bbox_h', 'score']

=== 1. 기본 통계 ===
총 예측 수: 3307
처리된 이미지 수: 843
탐지된 카테고리 수: 73
평균 신뢰도: 0.543
신뢰도 범위: 0.250 ~ 0.773

=== 2. 신뢰도 분포 분석 ===
count    3307.000000
mean        0.542555
std         0.092760
min         0.250021
25%         0.498815
50%         0.553276
75%         0.604353
max         0.772695
Name: confidence, dtype: float64

신뢰도 구간별 예측 수:
  (-0.001, 0.25]: 0개 (0.0%)
  (0.25, 0.3]: 94개 (2.8%)
  (0.3, 0.35]: 87개 (2.6%)
  (0.35, 0.4]: 81개 (2.4%)
  (0.4, 0.5]: 576개 (17.4%)
  (0.5, 0.6]: 1581개 (47.8%)
  (0.6, 0.7]: 843개 (25.5%)
  (0.7, 0.8]: 45개 (1.4%)
  (0.8, 1.0]: 0개 (0.0%)

=== 3. 카테고리별 분석 ===
상위 10개 카테고리:
             예측수  평균신뢰도  신뢰도표준편차  이미지수
category_id                           
3482         370  0.544    0.025   369
2482         138  0.481    0.039   134
3350         137  0.591    0.069   137
1899         126  0.600    0.034  

In [5]:
def analyze_misclassification_patterns(csv_path="submission_baseline_return.csv"):
    """오분류 패턴 중심 분석 - 성능 향상의 핵심"""

    print("=== 🎯 오분류 중심 분석 시작 ===\n")

    # 데이터 로드
    df = pd.read_csv(csv_path)
    df['box_area'] = df['bbox_w'] * df['bbox_h']

    print(f"분석 대상: {len(df)}개 예측")

    # 1. 잠재적 오분류 후보 식별
    print("=== 1. 잠재적 오분류 후보 식별 ===")

    # 1-1. 신뢰도 기반 의심 구간
    low_confidence = df[df['score'] < 0.4]
    medium_confidence = df[(df['score'] >= 0.4) & (df['score'] < 0.6)]
    high_confidence = df[df['score'] >= 0.6]

    print(f"낮은 신뢰도 (<0.4): {len(low_confidence)}개 ({len(low_confidence)/len(df)*100:.1f}%)")
    print(f"중간 신뢰도 (0.4-0.6): {len(medium_confidence)}개 ({len(medium_confidence)/len(df)*100:.1f}%)")
    print(f"높은 신뢰도 (≥0.6): {len(high_confidence)}개 ({len(high_confidence)/len(df)*100:.1f}%)")

    # 1-2. 박스 크기 기반 의심 구간
    small_boxes = df[df['box_area'] < 10000]  # 너무 작은 박스
    large_boxes = df[df['box_area'] > 150000]  # 너무 큰 박스
    normal_boxes = df[(df['box_area'] >= 10000) & (df['box_area'] <= 150000)]

    print(f"\n박스 크기별 분포:")
    print(f"작은 박스 (<10k): {len(small_boxes)}개 ({len(small_boxes)/len(df)*100:.1f}%)")
    print(f"정상 박스 (10k-150k): {len(normal_boxes)}개 ({len(normal_boxes)/len(df)*100:.1f}%)")
    print(f"큰 박스 (>150k): {len(large_boxes)}개 ({len(large_boxes)/len(df)*100:.1f}%)")

    # 2. 카테고리별 오분류 가능성 분석
    print(f"\n=== 2. 카테고리별 오분류 위험도 분석 ===")

    category_analysis = df.groupby('category_id').agg({
        'score': ['count', 'mean', 'std', 'min'],
        'box_area': ['mean', 'std'],
        'image_id': 'nunique'
    }).round(3)

    category_analysis.columns = ['예측수', '평균신뢰도', '신뢰도편차', '최저신뢰도', '평균면적', '면적편차', '이미지수']

    # 오분류 위험도 점수 계산
    category_analysis['오분류위험도'] = (
        (category_analysis['신뢰도편차'] * 2) +  # 신뢰도 편차가 클수록 위험
        (1 - category_analysis['평균신뢰도']) +   # 평균 신뢰도가 낮을수록 위험
        (category_analysis['면적편차'] / 50000)   # 면적 편차가 클수록 위험
    ).round(3)

    # 위험도별 정렬
    high_risk_categories = category_analysis.sort_values('오분류위험도', ascending=False)

    print("상위 10개 오분류 위험 카테고리:")
    print(high_risk_categories[['예측수', '평균신뢰도', '신뢰도편차', '오분류위험도']].head(10))

    # 3. 혼동되기 쉬운 카테고리 쌍 식별
    print(f"\n=== 3. 혼동 가능 카테고리 분석 ===")

    # 같은 이미지 내에서 여러 카테고리가 탐지되는 패턴 분석
    image_category_counts = df.groupby('image_id')['category_id'].agg(['count', 'nunique'])
    multi_category_images = image_category_counts[image_category_counts['nunique'] > 3]  # 4개 이상 카테고리

    print(f"다중 카테고리 이미지 (4개 이상): {len(multi_category_images)}개")

    if len(multi_category_images) > 0:
        # 이런 이미지들에서 어떤 카테고리들이 함께 나타나는지 분석
        multi_category_data = df[df['image_id'].isin(multi_category_images.index)]
        category_cooccurrence = defaultdict(int)

        for image_id in multi_category_images.index:
            image_categories = df[df['image_id'] == image_id]['category_id'].unique()
            for i, cat1 in enumerate(image_categories):
                for cat2 in image_categories[i+1:]:
                    pair = tuple(sorted([cat1, cat2]))
                    category_cooccurrence[pair] += 1

        # 가장 자주 함께 나타나는 카테고리 쌍들
        frequent_pairs = sorted(category_cooccurrence.items(), key=lambda x: x[1], reverse=True)[:10]
        print("자주 함께 탐지되는 카테고리 쌍 (혼동 가능):")
        for (cat1, cat2), count in frequent_pairs:
            print(f"  카테고리 {cat1} - {cat2}: {count}회")

    # 4. 공간적 오분류 패턴
    print(f"\n=== 4. 공간적 오분류 패턴 ===")

    # 박스 위치별 신뢰도 분석 (이미지 내 위치)
    df['bbox_center_x'] = df['bbox_x'] + df['bbox_w'] / 2
    df['bbox_center_y'] = df['bbox_y'] + df['bbox_h'] / 2

    # 이미지 크기를 1000x1000으로 가정하고 구역별 분석
    df['region_x'] = pd.cut(df['bbox_center_x'], bins=3, labels=['왼쪽', '중앙', '오른쪽'])
    df['region_y'] = pd.cut(df['bbox_center_y'], bins=3, labels=['상단', '중간', '하단'])

    region_confidence = df.groupby(['region_x', 'region_y'])['score'].agg(['count', 'mean']).round(3)
    print("이미지 영역별 신뢰도:")
    print(region_confidence)

    # 5. 오분류 제거 전략 제안
    print(f"\n=== 5. 🎯 오분류 제거 전략 ===")

    # 전략 1: 고위험 카테고리 + 낮은 신뢰도
    high_risk_cats = high_risk_categories.head(5).index.tolist()
    risky_predictions = df[
        (df['category_id'].isin(high_risk_cats)) &
        (df['score'] < 0.45)
    ]

    print(f"전략 1 - 고위험 카테고리 + 낮은 신뢰도 제거:")
    print(f"  제거 대상: {len(risky_predictions)}개 ({len(risky_predictions)/len(df)*100:.1f}%)")
    print(f"  남은 예측: {len(df) - len(risky_predictions)}개")

    # 전략 2: 큰 박스 + 낮은 신뢰도
    large_low_conf = df[
        (df['box_area'] > 150000) &
        (df['score'] < 0.5)
    ]

    print(f"\n전략 2 - 큰 박스 + 낮은 신뢰도 제거:")
    print(f"  제거 대상: {len(large_low_conf)}개 ({len(large_low_conf)/len(df)*100:.1f}%)")
    print(f"  남은 예측: {len(df) - len(large_low_conf)}개")

    # 전략 3: 종합적 오분류 스코어링
    df['misclassification_score'] = (
        (1 - df['score']) * 0.4 +  # 낮은 신뢰도일수록 높은 점수
        (df['box_area'] > 150000).astype(int) * 0.3 +  # 큰 박스일수록 높은 점수
        (df['category_id'].isin(high_risk_cats)).astype(int) * 0.3  # 고위험 카테고리일수록 높은 점수
    )

    # 상위 10% 오분류 의심 예측들
    threshold_90 = df['misclassification_score'].quantile(0.9)
    suspected_misclassifications = df[df['misclassification_score'] >= threshold_90]

    print(f"\n전략 3 - 종합 오분류 점수 기반 (상위 10% 제거):")
    print(f"  제거 대상: {len(suspected_misclassifications)}개 ({len(suspected_misclassifications)/len(df)*100:.1f}%)")
    print(f"  남은 예측: {len(df) - len(suspected_misclassifications)}개")
    print(f"  오분류 점수 범위: {suspected_misclassifications['misclassification_score'].min():.3f} ~ {suspected_misclassifications['misclassification_score'].max():.3f}")

    # 6. 구체적 실행 코드 제안
    print(f"\n=== 6. 💻 즉시 실행 가능한 개선 코드 ===")

    print("# 가장 안전한 오분류 제거 (전략 2)")
    print(f"# 큰 박스 + 낮은 신뢰도 제거로 {len(large_low_conf)}개 제거")
    print("filtered_df = df[~((df['box_area'] > 150000) & (df['score'] < 0.5))]")

    print("\n# 더 공격적인 오분류 제거 (전략 3)")
    print(f"# 종합 점수 기반으로 상위 10% 제거하여 {len(suspected_misclassifications)}개 제거")
    print("filtered_df = df[df['misclassification_score'] < df['misclassification_score'].quantile(0.9)]")

    return df, high_risk_categories, suspected_misclassifications

def create_filtered_submission(df, strategy="safe", output_path="submission_filtered.csv"):
    """오분류 제거 후 새로운 제출 파일 생성"""

    print(f"\n=== 오분류 제거 실행 ({strategy}) ===")

    original_count = len(df)

    if strategy == "safe":
        # 안전한 전략: 큰 박스 + 낮은 신뢰도만 제거
        filtered_df = df[~((df['box_area'] > 150000) & (df['score'] < 0.5))]

    elif strategy == "aggressive":
        # 공격적 전략: 종합 오분류 점수 상위 10% 제거
        df['misclassification_score'] = (
            (1 - df['score']) * 0.4 +
            (df['box_area'] > 150000).astype(int) * 0.3 +
            (df['category_id'].isin([3482, 2482, 3350])).astype(int) * 0.3  # 상위 3개 카테고리
        )
        threshold = df['misclassification_score'].quantile(0.9)
        filtered_df = df[df['misclassification_score'] < threshold]

    elif strategy == "moderate":
        # 중간 전략: 복합 조건
        filtered_df = df[~(
            ((df['box_area'] > 150000) & (df['score'] < 0.5)) |  # 큰 박스 + 낮은 신뢰도
            ((df['score'] < 0.3)) |  # 매우 낮은 신뢰도
            ((df['box_area'] > 200000))  # 매우 큰 박스
        )]

    else:
        print("전략을 선택하세요: 'safe', 'moderate', 'aggressive'")
        return None

    removed_count = original_count - len(filtered_df)

    print(f"제거된 예측: {removed_count}개 ({removed_count/original_count*100:.1f}%)")
    print(f"남은 예측: {len(filtered_df)}개")
    print(f"평균 신뢰도 변화: {df['score'].mean():.3f} → {filtered_df['score'].mean():.3f}")

    # 새로운 제출 파일 생성
    filtered_df.to_csv(output_path, index=False)
    print(f"필터링된 제출 파일 저장: {output_path}")

    return filtered_df

# 메인 실행 함수
def run_misclassification_analysis(csv_path="submission_baseline_return.csv"):
    """오분류 분석 전체 실행"""

    # 오분류 패턴 분석
    df, high_risk_categories, suspected_misclassifications = analyze_misclassification_patterns(csv_path)

    # 3가지 전략으로 필터링된 파일 생성
    safe_df = create_filtered_submission(df, "safe", "submission_safe_filtered.csv")
    moderate_df = create_filtered_submission(df, "moderate", "submission_moderate_filtered.csv")
    aggressive_df = create_filtered_submission(df, "aggressive", "submission_aggressive_filtered.csv")

    print(f"\n=== 📊 전략별 요약 ===")
    print(f"원본: {len(df)}개")
    print(f"Safe: {len(safe_df)}개 (-{len(df)-len(safe_df)}개)")
    print(f"Moderate: {len(moderate_df)}개 (-{len(df)-len(moderate_df)}개)")
    print(f"Aggressive: {len(aggressive_df)}개 (-{len(df)-len(aggressive_df)}개)")

    print(f"\n🎯 추천: Safe 전략부터 시작하여 성능 확인!")

    return df, safe_df, moderate_df, aggressive_df

In [8]:
if __name__ == "__main__":
    # 오분류 중심 분석 실행
    run_misclassification_analysis("./submission_baseline_return.csv")

=== 🎯 오분류 중심 분석 시작 ===

분석 대상: 3307개 예측
=== 1. 잠재적 오분류 후보 식별 ===
낮은 신뢰도 (<0.4): 262개 (7.9%)
중간 신뢰도 (0.4-0.6): 2157개 (65.2%)
높은 신뢰도 (≥0.6): 888개 (26.9%)

박스 크기별 분포:
작은 박스 (<10k): 0개 (0.0%)
정상 박스 (10k-150k): 2892개 (87.5%)
큰 박스 (>150k): 415개 (12.5%)

=== 2. 카테고리별 오분류 위험도 분석 ===
상위 10개 오분류 위험 카테고리:
             예측수  평균신뢰도  신뢰도편차  오분류위험도
category_id                           
3831          21  0.462  0.171   1.933
18146         25  0.421  0.126   1.230
16687         19  0.575  0.061   1.200
20013         14  0.442  0.059   1.109
22626         36  0.521  0.131   0.973
31884         63  0.457  0.051   0.945
25366         56  0.653  0.071   0.913
34596         76  0.541  0.125   0.911
5885          39  0.457  0.075   0.899
27992          7  0.545  0.119   0.896

=== 3. 혼동 가능 카테고리 분석 ===
다중 카테고리 이미지 (4개 이상): 691개
자주 함께 탐지되는 카테고리 쌍 (혼동 가능):
  카테고리 3482 - 27652: 72회
  카테고리 1899 - 16550: 68회
  카테고리 3482 - 30307: 66회
  카테고리 3482 - 27776: 65회
  카테고리 3482 - 25468: 64회
  카테고리 3482 - 16231: 63회
  카테고리 

/tmp/ipython-input-3677387173.py:96: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  region_confidence = df.groupby(['region_x', 'region_y'])['score'].agg(['count', 'mean']).round(3)


필터링된 제출 파일 저장: submission_aggressive_filtered.csv

=== 📊 전략별 요약 ===
원본: 3307개
Safe: 3208개 (-99개)
Moderate: 3089개 (-218개)
Aggressive: 2976개 (-331개)

🎯 추천: Safe 전략부터 시작하여 성능 확인!


In [9]:
import pandas as pd
import numpy as np

def apply_targeted_misclassification_removal(csv_path="submission_baseline_return.csv"):
    """분석 결과 기반 타겟팅된 오분류 제거"""

    print("=== 🎯 타겟팅된 오분류 제거 실행 ===\n")

    # 데이터 로드
    df = pd.read_csv(csv_path)
    df['box_area'] = df['bbox_w'] * df['bbox_h']
    original_count = len(df)

    print(f"원본 예측 수: {original_count}개")
    print(f"평균 신뢰도: {df['score'].mean():.3f}")

    # 분석에서 확인된 고위험 요소들
    high_risk_categories = [3831, 18146, 16687, 20013, 22626]  # 상위 5개 고위험 카테고리
    frequent_confusion_category = 3482  # 가장 많은 혼동을 일으키는 카테고리

    # 전략 1: 큰 박스 + 낮은 신뢰도 제거 (가장 안전)
    def strategy_1_large_boxes(df):
        """큰 박스 + 낮은 신뢰도 제거"""
        condition = (df['box_area'] > 150000) & (df['score'] < 0.5)
        removed = df[condition]
        filtered = df[~condition]

        print("=== 전략 1: 큰 박스 정화 ===")
        print(f"제거 조건: 박스 면적 > 150,000 AND 신뢰도 < 0.5")
        print(f"제거된 예측: {len(removed)}개 ({len(removed)/len(df)*100:.1f}%)")
        print(f"남은 예측: {len(filtered)}개")
        print(f"신뢰도 향상: {df['score'].mean():.3f} → {filtered['score'].mean():.3f}")

        return filtered, removed

    # 전략 2: 고위험 카테고리 정화
    def strategy_2_high_risk_categories(df):
        """고위험 카테고리의 낮은 신뢰도 제거"""
        condition = (df['category_id'].isin(high_risk_categories)) & (df['score'] < 0.45)
        removed = df[condition]
        filtered = df[~condition]

        print("\n=== 전략 2: 고위험 카테고리 정화 ===")
        print(f"고위험 카테고리: {high_risk_categories}")
        print(f"제거 조건: 고위험 카테고리 AND 신뢰도 < 0.45")
        print(f"제거된 예측: {len(removed)}개 ({len(removed)/len(df)*100:.1f}%)")
        print(f"남은 예측: {len(filtered)}개")
        print(f"신뢰도 향상: {df['score'].mean():.3f} → {filtered['score'].mean():.3f}")

        return filtered, removed

    # 전략 3: 과분류 카테고리 억제
    def strategy_3_suppress_overclassification(df):
        """카테고리 3482의 과분류 억제"""
        condition = (df['category_id'] == frequent_confusion_category) & (df['score'] < 0.4)
        removed = df[condition]
        filtered = df[~condition]

        print(f"\n=== 전략 3: 과분류 억제 (카테고리 {frequent_confusion_category}) ===")
        print(f"제거 조건: 카테고리 {frequent_confusion_category} AND 신뢰도 < 0.4")
        print(f"제거된 예측: {len(removed)}개 ({len(removed)/len(df)*100:.1f}%)")
        print(f"남은 예측: {len(filtered)}개")
        print(f"신뢰도 향상: {df['score'].mean():.3f} → {filtered['score'].mean():.3f}")

        return filtered, removed

    # 전략 4: 종합적 정화 (보수적)
    def strategy_4_conservative_combined(df):
        """보수적 종합 정화"""
        condition = (
            # 큰 박스 + 낮은 신뢰도
            ((df['box_area'] > 150000) & (df['score'] < 0.5)) |
            # 고위험 카테고리 + 매우 낮은 신뢰도
            ((df['category_id'].isin(high_risk_categories)) & (df['score'] < 0.4)) |
            # 전체적으로 매우 낮은 신뢰도
            (df['score'] < 0.3)
        )

        removed = df[condition]
        filtered = df[~condition]

        print(f"\n=== 전략 4: 보수적 종합 정화 ===")
        print(f"제거 조건:")
        print(f"  1) 큰 박스(>150k) + 낮은신뢰도(<0.5)")
        print(f"  2) 고위험 카테고리 + 매우낮은신뢰도(<0.4)")
        print(f"  3) 전체 매우낮은신뢰도(<0.3)")
        print(f"제거된 예측: {len(removed)}개 ({len(removed)/len(df)*100:.1f}%)")
        print(f"남은 예측: {len(filtered)}개")
        print(f"신뢰도 향상: {df['score'].mean():.3f} → {filtered['score'].mean():.3f}")

        return filtered, removed

    # 전략 5: 적극적 정화
    def strategy_5_aggressive_combined(df):
        """적극적 종합 정화"""
        condition = (
            # 모든 큰 박스
            (df['box_area'] > 130000) |
            # 고위험 카테고리 + 중간 신뢰도
            ((df['category_id'].isin(high_risk_categories)) & (df['score'] < 0.5)) |
            # 과분류 카테고리 + 낮은 신뢰도
            ((df['category_id'] == frequent_confusion_category) & (df['score'] < 0.45)) |
            # 전체적으로 낮은 신뢰도
            (df['score'] < 0.35)
        )

        removed = df[condition]
        filtered = df[~condition]

        print(f"\n=== 전략 5: 적극적 종합 정화 ===")
        print(f"제거 조건:")
        print(f"  1) 큰 박스(>130k)")
        print(f"  2) 고위험 카테고리 + 중간신뢰도(<0.5)")
        print(f"  3) 과분류 카테고리({frequent_confusion_category}) + 낮은신뢰도(<0.45)")
        print(f"  4) 전체 낮은신뢰도(<0.35)")
        print(f"제거된 예측: {len(removed)}개 ({len(removed)/len(df)*100:.1f}%)")
        print(f"남은 예측: {len(filtered)}개")
        print(f"신뢰도 향상: {df['score'].mean():.3f} → {filtered['score'].mean():.3f}")

        return filtered, removed

    # 모든 전략 실행
    results = {}

    # 전략 1 (가장 안전)
    filtered_1, removed_1 = strategy_1_large_boxes(df.copy())
    filtered_1.to_csv("submission_strategy1_large_boxes.csv", index=False)
    results['strategy_1'] = (filtered_1, removed_1)

    # 전략 2
    filtered_2, removed_2 = strategy_2_high_risk_categories(df.copy())
    filtered_2.to_csv("submission_strategy2_high_risk.csv", index=False)
    results['strategy_2'] = (filtered_2, removed_2)

    # 전략 3
    filtered_3, removed_3 = strategy_3_suppress_overclassification(df.copy())
    filtered_3.to_csv("submission_strategy3_suppress_overclass.csv", index=False)
    results['strategy_3'] = (filtered_3, removed_3)

    # 전략 4 (보수적 종합)
    filtered_4, removed_4 = strategy_4_conservative_combined(df.copy())
    filtered_4.to_csv("submission_strategy4_conservative.csv", index=False)
    results['strategy_4'] = (filtered_4, removed_4)

    # 전략 5 (적극적 종합)
    filtered_5, removed_5 = strategy_5_aggressive_combined(df.copy())
    filtered_5.to_csv("submission_strategy5_aggressive.csv", index=False)
    results['strategy_5'] = (filtered_5, removed_5)

    # 결과 요약
    print(f"\n=== 📊 전략별 결과 요약 ===")
    print(f"{'전략':<20} {'제거수':<8} {'제거율':<8} {'남은예측':<8} {'신뢰도향상':<12}")
    print("-" * 60)

    strategies = [
        ("Strategy 1 (큰박스)", filtered_1, removed_1),
        ("Strategy 2 (고위험)", filtered_2, removed_2),
        ("Strategy 3 (과분류)", filtered_3, removed_3),
        ("Strategy 4 (보수적)", filtered_4, removed_4),
        ("Strategy 5 (적극적)", filtered_5, removed_5)
    ]

    for name, filtered, removed in strategies:
        remove_pct = len(removed) / original_count * 100
        conf_improvement = filtered['score'].mean() - df['score'].mean()
        print(f"{name:<20} {len(removed):<8} {remove_pct:<7.1f}% {len(filtered):<8} {conf_improvement:<+11.3f}")

    print(f"\n=== 🎯 추천 사항 ===")
    print(f"1순위: Strategy 1 (큰 박스) - 가장 안전하고 확실한 개선")
    print(f"2순위: Strategy 4 (보수적) - 균형잡힌 종합 접근")
    print(f"3순위: Strategy 2 (고위험) - 타겟팅된 카테고리 정화")

    print(f"\n생성된 파일들:")
    print(f"  - submission_strategy1_large_boxes.csv")
    print(f"  - submission_strategy2_high_risk.csv")
    print(f"  - submission_strategy3_suppress_overclass.csv")
    print(f"  - submission_strategy4_conservative.csv")
    print(f"  - submission_strategy5_aggressive.csv")

    return results

def analyze_removed_predictions(df_original, df_filtered, removed_df):
    """제거된 예측들의 특성 분석"""

    print(f"\n=== 🔍 제거된 예측 분석 ===")

    if len(removed_df) == 0:
        print("제거된 예측이 없습니다.")
        return

    print(f"제거된 예측 수: {len(removed_df)}개")
    print(f"제거된 예측의 평균 신뢰도: {removed_df['score'].mean():.3f}")
    print(f"제거된 예측의 신뢰도 범위: {removed_df['score'].min():.3f} ~ {removed_df['score'].max():.3f}")

    # 제거된 카테고리별 분포
    removed_categories = removed_df['category_id'].value_counts().head(10)
    print(f"\n제거된 예측의 카테고리별 분포 (상위 10개):")
    for cat_id, count in removed_categories.items():
        pct = count / len(removed_df) * 100
        print(f"  카테고리 {cat_id}: {count}개 ({pct:.1f}%)")

    # 박스 크기 분포
    if 'box_area' in removed_df.columns:
        avg_area = removed_df['box_area'].mean()
        print(f"\n제거된 예측의 평균 박스 크기: {avg_area:.0f}")
        large_boxes_removed = len(removed_df[removed_df['box_area'] > 150000])
        print(f"제거된 큰 박스 (>150k): {large_boxes_removed}개")

# 메인 실행 함수
def run_targeted_removal():
    """타겟팅된 오분류 제거 실행"""
    results = apply_targeted_misclassification_removal("./submission_baseline_return.csv")

    # 가장 추천하는 Strategy 1 결과 상세 분석
    if 'strategy_1' in results:
        filtered_1, removed_1 = results['strategy_1']
        analyze_removed_predictions(
            pd.read_csv("submission_baseline_return.csv"),
            filtered_1,
            removed_1
        )

    return results

if __name__ == "__main__":
    run_targeted_removal()

=== 🎯 타겟팅된 오분류 제거 실행 ===

원본 예측 수: 3307개
평균 신뢰도: 0.543
=== 전략 1: 큰 박스 정화 ===
제거 조건: 박스 면적 > 150,000 AND 신뢰도 < 0.5
제거된 예측: 99개 (3.0%)
남은 예측: 3208개
신뢰도 향상: 0.543 → 0.547

=== 전략 2: 고위험 카테고리 정화 ===
고위험 카테고리: [3831, 18146, 16687, 20013, 22626]
제거 조건: 고위험 카테고리 AND 신뢰도 < 0.45
제거된 예측: 39개 (1.2%)
남은 예측: 3268개
신뢰도 향상: 0.543 → 0.545

=== 전략 3: 과분류 억제 (카테고리 3482) ===
제거 조건: 카테고리 3482 AND 신뢰도 < 0.4
제거된 예측: 1개 (0.0%)
남은 예측: 3306개
신뢰도 향상: 0.543 → 0.543

=== 전략 4: 보수적 종합 정화 ===
제거 조건:
  1) 큰 박스(>150k) + 낮은신뢰도(<0.5)
  2) 고위험 카테고리 + 매우낮은신뢰도(<0.4)
  3) 전체 매우낮은신뢰도(<0.3)
제거된 예측: 187개 (5.7%)
남은 예측: 3120개
신뢰도 향상: 0.543 → 0.554

=== 전략 5: 적극적 종합 정화 ===
제거 조건:
  1) 큰 박스(>130k)
  2) 고위험 카테고리 + 중간신뢰도(<0.5)
  3) 과분류 카테고리(3482) + 낮은신뢰도(<0.45)
  4) 전체 낮은신뢰도(<0.35)
제거된 예측: 745개 (22.5%)
남은 예측: 2562개
신뢰도 향상: 0.543 → 0.556

=== 📊 전략별 결과 요약 ===
전략                   제거수      제거율      남은예측     신뢰도향상       
------------------------------------------------------------
Strategy 1 (큰박스)     99       3.0    % 3208     +0.004 